In [3]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
import re
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
# Extract text from inside

def preprocess(folder):
    no_of_paras = []
    current_dir = directory + "/"+folder
    base_cases = []
    noticed_cases = []
    paragraphs = []
    entailed_fragments = []
    list_of_files = sorted(os.listdir(current_dir))
    base_judgements = []
    candidate_judgements = []
    x = 1
    y = len(list_of_files)-x
    for i in range(y):
        print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        g = open(current_dir+'/'+list_of_files[i]+"/noticed_cases.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/candidates"))
        a = f.read()
        b = g.read()
        base_cases.append(a)
        tmp = re.search("\n\nJUDGMENT",a).start()
        noticed_cases.append(list(map(int,b.split("\n")[:-1])))
        f.close()
       # g.close()
        
        arr = []
        no_of_paras.append(len(paragraph_filenames))

        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/candidates/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,no_of_paras,list_of_files,noticed_cases
    
train_base_cases,train_entailed_fragments, train_paragraphs,train_no_of_paras,train_list_of_files,train_noticed_cases = preprocess('task1_train')
    
#test_base_cases,test_entailed_fragments, test_paragraphs,test_no_of_paras = preprocess('task1_test')

Processing  0


AttributeError: 'NoneType' object has no attribute 'start'

In [ ]:
train_noticed_cases[0][:50]

In [7]:
data = train_base_cases + train_paragraphs

In [5]:
def gen_labels(data):
    labels = []
    for i in range(len(train_base_cases)):
        labels.append("base_case_"+train_list_of_files[i])
    for i in range(len(train_base_cases)):
        for j in range(200):
            labels.append("candidate_"+train_list_of_files[i]+"_"+str(j+1))
    return labels
labels = gen_labels(data)

In [ ]:
import os
from os import listdir
from os.path import isfile, join
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
import gensim, logging
import time
import re
import nltk
import io
from nltk.corpus import stopwords
import numpy as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())

In [ ]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=re.split('\W+',doc), tags=[self.labels_list[idx]])
iterator = DocIterator(data, labels)

In [ ]:
model = gensim.models.Doc2Vec(vector_size=150, window=20, min_count=3, workers=4, alpha=0.025, min_alpha=0.001)
model.build_vocab(iterator)

print('done building vocabulary')
print('start training the model')
tic = time.time()
model.train(iterator,total_examples=model.corpus_count, epochs = 10)
toc = time.time()
print(toc-tic)
model.save("doc2vec_task_1.model")

In [3]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("doc2vec_task_1.model")

2019-03-14 20:31:35,843 : INFO : loading Doc2Vec object from doc2vec_task_1.model
2019-03-14 20:31:37,187 : INFO : loading vocabulary recursively from doc2vec_task_1.model.vocabulary.* with mmap=None
2019-03-14 20:31:37,188 : INFO : loading trainables recursively from doc2vec_task_1.model.trainables.* with mmap=None
2019-03-14 20:31:37,189 : INFO : loading syn1neg from doc2vec_task_1.model.trainables.syn1neg.npy with mmap=None
2019-03-14 20:31:37,968 : INFO : loading wv recursively from doc2vec_task_1.model.wv.* with mmap=None
2019-03-14 20:31:37,969 : INFO : loading vectors from doc2vec_task_1.model.wv.vectors.npy with mmap=None
2019-03-14 20:31:38,762 : INFO : loading docvecs recursively from doc2vec_task_1.model.docvecs.* with mmap=None
2019-03-14 20:31:38,764 : INFO : loaded doc2vec_task_1.model


In [ ]:
model.docvecs.most_similar(labels[3],topn=500)

In [32]:
correct_pred = 0
total_pred = 0
total_true = 0
for i in range(285):
    print("\nPredicting  For ",i)
    lst = []
    for j in range(200):
        #lst.append((model.docvecs.similarity(labels[i],labels[285+i*200+j]),j+1))
        lst.append((model.docvecs.similarity(labels[i],"candidate_"+train_list_of_files[i]+"_"+str(j+1)),j+1))
    lst.sort(reverse=True)
    total_true += len(train_noticed_cases[i])
    print(train_noticed_cases[i])
    for j in range(10):
        if j>=3:
            if lst[j][0] < 0.55:
                print("Breaking")
                break
        total_pred += 1
        if lst[j][1] in train_noticed_cases[i]:
            correct_pred += 1
            print("predicted correct",j+1,lst[j][0])
        #else:
            #print("Wrong",j+1,lst[j][0])


Predicting  For  0
[58, 78, 102, 104]
predicted correct 1 0.5558144
Breaking

Predicting  For  1
[10, 151]
Breaking

Predicting  For  2
[138, 143]
predicted correct 2 0.53457713
Breaking

Predicting  For  3
[68, 74, 112, 152, 184]
Breaking

Predicting  For  4
[1, 17, 78, 91, 95, 114, 158]
predicted correct 1 0.5520998
predicted correct 3 0.50852716
Breaking

Predicting  For  5
[35, 91]
predicted correct 1 0.6919823
predicted correct 2 0.45378083
Breaking

Predicting  For  6
[49, 53, 65, 175]
predicted correct 2 0.5816579
Breaking

Predicting  For  7
[28, 32, 73, 76, 87, 122, 167, 170]
predicted correct 1 0.73567396
predicted correct 2 0.7294586

Predicting  For  8
[28, 108, 152]

Predicting  For  9
[28, 39, 63]
predicted correct 1 0.72565365
predicted correct 2 0.65890235
predicted correct 3 0.6574016
Breaking

Predicting  For  10
[4, 35, 47, 92, 186]
predicted correct 1 0.6620822
predicted correct 2 0.6461615
predicted correct 3 0.6271997

Predicting  For  11
[79, 178]
predicted corr

[109]
predicted correct 10 0.55112594

Predicting  For  120
[2]
predicted correct 1 0.56119347
Breaking

Predicting  For  121
[42, 58, 77, 129, 167]
predicted correct 1 0.6176926
predicted correct 2 0.5660665
predicted correct 3 0.5304748
Breaking

Predicting  For  122
[87, 93, 97, 162, 186]

Predicting  For  123
[58, 69, 94, 159, 180]
predicted correct 1 0.5431587
predicted correct 2 0.53388715
Breaking

Predicting  For  124
[32]
predicted correct 1 0.60803694
Breaking

Predicting  For  125
[13, 35, 64, 66, 81, 122, 129, 137, 159, 164, 165, 170, 171]
predicted correct 2 0.7153681
predicted correct 8 0.6898403

Predicting  For  126
[63, 163, 171, 196]
Breaking

Predicting  For  127
[38, 68, 188]
predicted correct 7 0.56573486

Predicting  For  128
[137]
predicted correct 1 0.569503
Breaking

Predicting  For  129
[48, 81, 90, 108]
predicted correct 1 0.54679143
predicted correct 2 0.53288937
Breaking

Predicting  For  130
[46]
predicted correct 1 0.58588696
Breaking

Predicting  For  13

[31, 94, 99, 104, 112, 136, 143, 164]
predicted correct 1 0.6551955
predicted correct 3 0.59874946
predicted correct 4 0.58898056
predicted correct 6 0.5737793
Breaking

Predicting  For  196
[23, 36, 49, 112]
predicted correct 1 0.5970945
predicted correct 2 0.53663754
Breaking

Predicting  For  197
[66, 68, 111, 172]

Predicting  For  198
[84, 86, 141]
predicted correct 1 0.53133345
predicted correct 2 0.52270085
Breaking

Predicting  For  199
[78, 150]
predicted correct 6 0.5669736
Breaking

Predicting  For  200
[9, 10, 11, 12, 72, 99, 107, 184]

Predicting  For  201
[57, 66, 102, 137]
predicted correct 2 0.4952109
Breaking

Predicting  For  202
[67, 153]

Predicting  For  203
[20, 58, 112]

Predicting  For  204
[30, 43, 104]
predicted correct 1 0.68561834
predicted correct 2 0.6554872
predicted correct 3 0.57367104
Breaking

Predicting  For  205
[42, 46, 109, 140, 145, 155, 192]
predicted correct 1 0.60357815
Breaking

Predicting  For  206
[32, 48, 106, 118]
Breaking

Predicting  Fo

In [33]:
correct_pred

468

In [12]:
total_true

1486

In [34]:
total_pred

1684

In [30]:
588/(1486)

0.39569313593539707

In [31]:
588/2421

0.24287484510532836

In [35]:
468/1486

0.3149394347240915

In [ ]:
Precision = 0.37
Recall = 0.52

In [6]:
import re
a = re.search("Baban","I am Baban Gain Baban GAin").start()
print(a)

5


In [5]:
a['span']

IndexError: no such group